# CFBD Ingestion Walkthrough

This notebook demonstrates how to pull CollegeFootballData.com content from the production REST API, the experimental Next API, and optionally the GraphQL endpoint (requires Patreon Tier 3+), then persist sanitized extracts for the Script Ohio 2.0 modeling pipeline. Keep your `CFBD_API_KEY` in the environment before running any cells. Steps covered:

1. Configure shared paths and imports for the new `data_sources` and `features` modules.
2. Instantiate REST (production + Next) clients with automatic rate limiting. GraphQL client is optional.
3. Fetch sample games for a target season/week and preview the payloads.
4. **Compare performance** between REST and GraphQL methods.
5. Convert raw payloads into the 86-feature schema using `CFBDFeatureEngineer`.
6. Persist CSV/JSON outputs into the git-ignored `outputs/` folder for downstream agents or notebooks.

Note: GraphQL access requires Patreon Tier 3+ access. The notebook will work with REST API only if GraphQL is not available.



In [ ]:
from __future__ import annotations

import json
import os
import sys
import time
from datetime import datetime, timezone
from pathlib import Path

import pandas as pd

PROJECT_ROOT = Path.cwd().resolve().parents[1]
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from data_sources import (  # noqa: E402
    CFBDClientConfig,
    CFBDRESTDataSource,
)

# GraphQL client import is optional - requires Patreon Tier 3+ access
try:
    from data_sources import CFBDGraphQLClient  # noqa: E402
    graphql_available = True
except ImportError:
    CFBDGraphQLClient = None  # type: ignore
    graphql_available = False
    print("⚠️ GraphQL client not available - requires Patreon Tier 3+ access")

from features.cfbd_feature_engineering import (  # noqa: E402
    CFBDFeatureEngineer,
    FeatureEngineeringConfig,
)

print(f"Project root: {PROJECT_ROOT}")


In [ ]:
season = int(os.environ.get("CFBD_SAMPLE_SEASON", 2025))
week = int(os.environ.get("CFBD_SAMPLE_WEEK", 12))
team_filter = os.environ.get("CFBD_SAMPLE_TEAM")
api_key = os.environ.get("CFBD_API_KEY")
if not api_key:
    raise RuntimeError("CFBD_API_KEY must be set before running this notebook.")

# Initialize REST clients (FALLBACK)
rest_client = CFBDRESTDataSource(
    config=CFBDClientConfig(api_key=api_key, host="production")
)
next_client = CFBDRESTDataSource(
    config=CFBDClientConfig(api_key=api_key, host="next")
)

# GraphQL client initialization (PRIMARY METHOD)
use_graphql = False
graphql_client = None

if graphql_available and CFBDGraphQLClient is not None:
    try:
        graphql_client = CFBDGraphQLClient(api_key=api_key, host="production")
        # Test connectivity with simple query
        connectivity_test = graphql_client.query("query { __typename }", {})
        if connectivity_test:
            use_graphql = True
            print("✅ GraphQL client initialized (PRIMARY METHOD)")
            print("✅ GraphQL connectivity test passed")
        else:
            print("⚠️ GraphQL connectivity test failed - using REST as fallback")
            use_graphql = False
    except Exception as e:
        print(f"⚠️ GraphQL client initialization failed: {e}")
        print("⚠️ Falling back to REST API (FALLBACK METHOD)")
        use_graphql = False
        graphql_client = None
else:
    graphql_client = None
    use_graphql = False
    print("⚠️ GraphQL client not available - using REST API (FALLBACK METHOD)")
    print("   (GraphQL requires Patreon Tier 3+ access)")

print(f"\nConfiguration:")
print(f"  Method: {'GraphQL (PRIMARY)' if use_graphql else 'REST (FALLBACK)'}")
print(f"  Season: {season}, Week: {week}, Team: {team_filter or 'ALL'}")


In [ ]:
# GraphQL Scoreboard Query (PRIMARY METHOD)
df_graphql_scoreboard = None

if use_graphql and graphql_client is not None:
    try:
        scoreboard_query = """
        query Scoreboard($season: Int!, $week: smallint) {
          game(
            where: {
              season: { _eq: $season }
              week: { _eq: $week }
            }
            order_by: { startDate: asc }
          ) {
            id
            season
            week
            seasonType
            startDate
            homeTeam
            awayTeam
            homePoints
            awayPoints
            completed
            venue
            homeConference
            awayConference
            neutralSite
            conferenceGame
          }
        }
        """
        
        variables = {"season": season, "week": week}
        result = graphql_client.query(scoreboard_query, variables)
        
        if result and "game" in result and result["game"]:
            games_data = result["game"]
            df_graphql_scoreboard = pd.json_normalize(games_data)
            
            # Rename columns from camelCase to snake_case
            column_mapping = {
                "homeTeam": "home_team",
                "awayTeam": "away_team",
                "homePoints": "home_points",
                "awayPoints": "away_points",
                "seasonType": "season_type",
                "startDate": "start_date",
                "homeConference": "home_conference",
                "awayConference": "away_conference",
                "neutralSite": "neutral_site",
                "conferenceGame": "conference_game"
            }
            df_graphql_scoreboard = df_graphql_scoreboard.rename(columns=column_mapping)
            
            # Convert start_date to datetime if present
            if "start_date" in df_graphql_scoreboard.columns:
                df_graphql_scoreboard["start_date"] = pd.to_datetime(df_graphql_scoreboard["start_date"])
            
            print(f"✅ Fetched {len(df_graphql_scoreboard)} games via GraphQL (PRIMARY METHOD)")
        else:
            print("⚠️ GraphQL query returned no games")
            df_graphql_scoreboard = None
    except Exception as e:
        print(f"⚠️ GraphQL scoreboard query failed: {e}")
        print("   Falling back to REST API")
        df_graphql_scoreboard = None
        use_graphql = False
else:
    print("⚠️ GraphQL not available - skipping GraphQL scoreboard query")


In [ ]:
# GraphQL Ratings Query (PRIMARY METHOD)
df_graphql_ratings = None

if use_graphql and graphql_client is not None:
    try:
        ratings_query = """
        query Ratings($year: smallint!) {
          ratings(
            where: {
              year: { _eq: $year }
            }
            order_by: { spOverall: desc }
          ) {
            team
            year
            conference
            elo
            fpi
            spOverall
            spOffense
            spDefense
          }
        }
        """
        
        variables = {"year": season}
        result = graphql_client.query(ratings_query, variables)
        
        if result and "ratings" in result and result["ratings"]:
            ratings_data = result["ratings"]
            df_graphql_ratings = pd.json_normalize(ratings_data)
            
            # Rename camelCase to snake_case
            column_mapping = {
                "spOverall": "sp_overall",
                "spOffense": "sp_offense",
                "spDefense": "sp_defense"
            }
            df_graphql_ratings = df_graphql_ratings.rename(columns=column_mapping)
            
            print(f"✅ Fetched {len(df_graphql_ratings)} team ratings via GraphQL")
        else:
            print("⚠️ GraphQL ratings query returned no data")
            df_graphql_ratings = None
    except Exception as e:
        print(f"⚠️ GraphQL ratings query failed: {e}")
        df_graphql_ratings = None
else:
    print("⚠️ GraphQL not available - skipping GraphQL ratings query")


In [ ]:
# Fetch REST data for comparison (FALLBACK METHOD)
rest_games = rest_client.fetch_games(year=season, week=week, team=team_filter)
next_games = next_client.fetch_games(year=season, week=week, team=team_filter)

# Select primary dataset: GraphQL first, REST fallback
method_used = "REST (FALLBACK)"

if use_graphql and df_graphql_scoreboard is not None and not df_graphql_scoreboard.empty:
    # Use GraphQL as primary
    df_scoreboard = df_graphql_scoreboard.copy()
    method_used = "GraphQL (PRIMARY)"
    # Convert DataFrame to dict format for compatibility
    games_list = df_scoreboard.to_dict('records')
    print(f"\n✅ Using GraphQL data (PRIMARY METHOD)")
else:
    # Fallback to REST
    df_scoreboard = pd.DataFrame(rest_games)
    games_list = rest_games
    method_used = "REST (FALLBACK)"
    print(f"\n⚠️ Using REST data (FALLBACK METHOD)")

print(f"\nComparison:")
print(f"  REST games: {len(rest_games)}")
print(f"  Next games: {len(next_games)}")
print(f"  GraphQL games: {len(df_graphql_scoreboard) if df_graphql_scoreboard is not None and not df_graphql_scoreboard.empty else 0}")
print(f"\nSelected method: {method_used}")
print(f"Selected games count: {len(games_list)}")

# Display sample from selected primary source
if games_list:
    print(f"\nSample from {method_used}:")
    sample = games_list[0] if isinstance(games_list[0], dict) else df_scoreboard.iloc[0].to_dict()
    print(json.dumps(sample, indent=2, default=str)[:500])


In [ ]:
# Comparing Fetch Speeds: REST vs GraphQL
print("\n--- Performance Comparison ---")

# Measure REST
start_time = time.time()
_ = rest_client.fetch_games(year=season, week=week)
rest_duration = time.time() - start_time
print(f"REST API Fetch: {rest_duration:.4f} seconds")

# Measure GraphQL (if available)
if use_graphql and graphql_client is not None:
    start_time = time.time()
    _ = graphql_client.query(scoreboard_query, variables)
    gql_duration = time.time() - start_time
    print(f"GraphQL API Fetch: {gql_duration:.4f} seconds")
    
    if gql_duration < rest_duration:
        print(f"🚀 GraphQL was {rest_duration / gql_duration:.1f}x faster")
    else:
        print(f"⚠️ GraphQL was slower (overhead or network variance)")
else:
    print("GraphQL not available for comparison")

In [ ]:
# Advanced Querying: Fetching Nested Recruiting Data via GraphQL
# This demonstrates fetching data that requires multiple REST calls in a single GraphQL query

if use_graphql and graphql_client is not None:
    print("\n--- Advanced GraphQL Query: Top Recruits by Team ---")
    
    recruiting_query = """
    query TeamRecruiting($year: smallint!, $limit: Int) {
      teamTalent(where: {year: {_eq: $year}}, limit: 5, order_by: {talent: desc}) {
        team {
          school
          conference
        }
        talent
        recruits(limit: 3, order_by: {rating: desc}) {
          name
          rating
          position {
            position
          }
        }
      }
    }
    """
    
    try:
        # Fetch top 5 talent teams and their top 3 recruits
        result = graphql_client.query(recruiting_query, {"year": season, "limit": 5})
        
        if result and "teamTalent" in result:
            for team_data in result["teamTalent"]:
                school = team_data["team"]["school"]
                talent = team_data["talent"]
                print(f"\n{school} (Talent: {talent})")
                for recruit in team_data.get("recruits", []):
                    pos = recruit["position"]["position"] if recruit["position"] else "??"
                    print(f"  - {recruit['name']} ({pos}): {recruit['rating']}")
        else:
            print("No recruiting data returned")
            
    except Exception as e:
        print(f"Advanced query failed: {e}")
else:
    print("GraphQL not available - skipping advanced query example")

In [ ]:
engineer = CFBDFeatureEngineer(
    FeatureEngineeringConfig(season=season, enforce_reference_schema=True)
)

# Process selected primary dataset
source_param = "graphql" if use_graphql and method_used == "GraphQL (PRIMARY)" else "rest"
primary_df = engineer.prepare_games_frame(games_list, source=source_param)
primary_features = engineer.build_feature_frame(primary_df)

# Also process REST data for comparison
rest_df = engineer.prepare_games_frame(rest_games, source="rest")
next_df = engineer.prepare_games_frame(next_games, source="rest")
rest_features = engineer.build_feature_frame(rest_df)
next_features = engineer.build_feature_frame(next_df)

print(f"\nFeature Engineering Results:")
print(f"  Primary ({method_used}): {primary_df.shape}")
print(f"  REST (comparison): {rest_df.shape}")
print(f"  Next (comparison): {next_df.shape}")
print(f"\n✅ Feature engineering completed using {method_used}")

primary_features.head()


In [ ]:
timestamp = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
output_dir = PROJECT_ROOT / "outputs" / "notebooks" / f"cfbd_ingestion_{timestamp}"
output_dir.mkdir(parents=True, exist_ok=True)

# Save primary features (from selected method)
primary_features.to_csv(output_dir / "primary_features.csv", index=False)

# Save comparison features (REST and Next)
rest_features.to_csv(output_dir / "rest_features.csv", index=False)
next_features.to_csv(output_dir / "next_features.csv", index=False)

method_info = f"Primary method: {method_used}"
print(f"\n✅ Wrote sanitized extracts to {output_dir}")
print(f"   {method_info}")
print(f"   Files saved: primary_features.csv, rest_features.csv, next_features.csv")


## Rate-Limit & Troubleshooting Notes

- The REST clients enforce a 0.17s delay for 6 req/sec rate limiting
- HTTP 429 responses indicate exceeding CFBD limits; retry with exponential backoff or pause the notebook
- GraphQL access requires Patreon Tier 3+ and is optional - the notebook works with REST API only if GraphQL is not available
- Ensure `CFBD_API_KEY` is set in your environment variables (requires restart if changed)
- All notebook outputs land in `outputs/notebooks/` and are git-ignored; delete old extracts to save disk space

